# SLM Constraint Functions Tutorial

This tutorial showcases the constraint functions used in the paper examples.

We cover three categories:
1. **Generation Templates**: Create music from scratch with specific styles
2. **Re-editing Functions**: Modify existing music (replace, retime)
3. **Additive Functions**: Add new elements to existing music


In [1]:
# Colab setup
import os
import sys

IS_COLAB = not os.path.exists("../slm")

if IS_COLAB:
    print("Running in Google Colab - setting up environment...")
    !git clone https://github.com/erl-j/superposed-language-modelling
    !pip install -e superposed-language-modelling
    sys.path.insert(0, '/content/superposed-language-modelling')
    print("Setup complete!")
else:
    print("Running locally - skipping setup")


Running locally - skipping setup


## Setup and Imports


In [2]:
import random
import inspect
import torch
import fractions
import symusic
torch.serialization.add_safe_globals([fractions.Fraction])

from slm.load_model_util import load_model
from slm.constraints.core import MusicalEventConstraint
from slm.constraints.templates import (
    unconditional,
    pentatonic_guitar,
    breakbeat,
    extreme_drums,
    strings_and_flute,
    triplet_drums,
)
from slm.constraints.re import replace, retime
from slm.constraints.addx import (
    add_percussion,
    add_snare_ghost_notes,
    add_tom_fill,
    add_locked_in_bassline,
)
from slm.util import preview_sm, sm_fix_overlap_notes
from slm.conversion_utils import sm_to_events

# Config
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"Device: {DEVICE}")


Device: cuda


## Load Model


In [3]:
print("Loading SLM model...")
model = load_model(model_type="slm_mixed", epochs=150, device=DEVICE)

N_EVENTS = model.tokenizer.config["max_notes"]

def generate_from_constraints(e, cfg=1.0, temperature=1.0, top_p=1.0):
    """Generate music from constraints."""
    mask = model.tokenizer.event_constraints_to_mask(e).to(DEVICE)
    out = model.generate(
        mask,
        temperature=temperature,
        top_p=top_p,
        constraint_cfg=cfg,
        tokens_per_step=1,
        order="random",
    )[0].argmax(-1)
    sm = model.tokenizer.decode(out)
    return sm_fix_overlap_notes(sm)

# Helper to create constraint objects
ec = lambda: MusicalEventConstraint(model.tokenizer)

print("Ready!")


Loading SLM model...
Loading model from /root/.cache/slm/slm_mixed_150epochs.ckpt...
Ready!


## 1. Generation Templates

These functions create music from scratch with specific characteristics.


In [ ]:
print("Source code for 'unconditional':")
print(inspect.getsource(unconditional))

constraints = unconditional(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=False,
    tag="pop",
    tempo=120,
)

print("\nGenerating unconditional music...")
sm = generate_from_constraints(constraints, cfg=1.0)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


Source code for 'unconditional':
def unconditional(e, ec, n_events, beat_range, pitch_range, drums, tag, tempo):
    e = [ec() for _ in range(n_events)]
    return e



TypeError: unconditional() missing 2 required positional arguments: 'tag' and 'tempo'

### Pentatonic Guitar


In [ ]:
print("Source code for 'pentatonic_guitar':")
print(inspect.getsource(pentatonic_guitar))

constraints = pentatonic_guitar(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=False,
    tag="pop",
    tempo=120,
)

print("\nGenerating pentatonic guitar...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Strings and Flute


In [ ]:
print("Source code for 'strings_and_flute':")
print(inspect.getsource(strings_and_flute))

constraints = strings_and_flute(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=False,
    tag="classical",
    tempo=120,
)

print("\nGenerating strings and flute...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Breakbeat


In [ ]:
print("Source code for 'breakbeat':")
print(inspect.getsource(breakbeat))

constraints = breakbeat(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="metal",
    tempo=95,
)

print("\nGenerating breakbeat...")
sm = generate_from_constraints(constraints, cfg=1.0)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Extreme Drums


In [ ]:
print("Source code for 'extreme_drums':")
print(inspect.getsource(extreme_drums))

constraints = extreme_drums(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="metal",
    tempo=75,
)

print("\nGenerating extreme drums...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Triplet Drums


In [ ]:
print("Source code for 'triplet_drums':")
print(inspect.getsource(triplet_drums))

constraints = triplet_drums(
    e=[],
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="pop",
    tempo=120,
)

print("\nGenerating triplet drums...")
sm = generate_from_constraints(constraints, cfg=1.0)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


## 2. Re-editing Functions

These functions modify existing music within a specified region.


### Load Source Music

First, let's load a source MIDI file to use for editing examples.


In [ ]:
# Load source MIDI
source_path = "../assets/drums_bass_piano.mid"
source_sm = symusic.Score(source_path)

print(f"Source has {source_sm.note_num()} notes")
preview_sm(source_sm)


### Replace


In [ ]:
print("Source code for 'replace':")
print(inspect.getsource(replace))

# Convert source to events
source_events = sm_to_events(source_sm, tag="pop", tokenizer=model.tokenizer)

# Replace the harmonic center (bars 2-6, pitches 48-74)
constraints = replace(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    tick_range=[96, 288],  # bars 2-6 at 24 ticks per beat
    pitch_range=[48, 74],
    drums=False,
    tag="pop",
    tempo=120,
)

print("\nGenerating with replace constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Retime


In [ ]:
print("Source code for 'retime':")
print(inspect.getsource(retime))

# Retime the entire piece
constraints = retime(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    tick_range=[0, 384],  # full 4 bars
    pitch_range=[0, 128],  # all pitches
    drums=False,
    tag="pop",
    tempo=120,
)

print("\nGenerating with retime constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


## 3. Additive Functions

These functions add new elements to existing music while keeping the original.


### Add Percussion


In [ ]:
print("Source code for 'add_percussion':")
print(inspect.getsource(add_percussion))

source_events = sm_to_events(source_sm, tag="pop", tokenizer=model.tokenizer)

constraints = add_percussion(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="pop",
    tempo=120,
)

print("\nGenerating with add_percussion constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Add Snare Ghost Notes


In [ ]:
print("Source code for 'add_snare_ghost_notes':")
print(inspect.getsource(add_snare_ghost_notes))

source_events = sm_to_events(source_sm, tag="pop", tokenizer=model.tokenizer)

constraints = add_snare_ghost_notes(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="pop",
    tempo=120,
)

print("\nGenerating with add_snare_ghost_notes constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Add Tom Fill


In [ ]:
print("Source code for 'add_tom_fill':")
print(inspect.getsource(add_tom_fill))

source_events = sm_to_events(source_sm, tag="pop", tokenizer=model.tokenizer)

constraints = add_tom_fill(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=True,
    tag="pop",
    tempo=120,
)

print("\nGenerating with add_tom_fill constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


### Add Locked-In Bassline


In [ ]:
print("Source code for 'add_locked_in_bassline':")
print(inspect.getsource(add_locked_in_bassline))

source_events = sm_to_events(source_sm, tag="pop", tokenizer=model.tokenizer)

constraints = add_locked_in_bassline(
    e=source_events,
    ec=ec,
    n_events=N_EVENTS,
    beat_range=[0, 16],
    pitch_range=[0, 128],
    drums=False,
    tag="pop",
    tempo=120,
)

print("\nGenerating with add_locked_in_bassline constraint...")
sm = generate_from_constraints(constraints, cfg=1.5)
print(f"Generated {sm.note_num()} notes")
preview_sm(sm)


## Comparing Different CFG Values

Let's compare how different constraint CFG values affect generation with one of the templates.


In [ ]:
print("Comparing CFG values for pentatonic_guitar...\n")

for cfg_value in [0.0, 0.5, 1.0, 1.5]:
    constraints = pentatonic_guitar(
        e=[],
        ec=ec,
        n_events=N_EVENTS,
        beat_range=[0, 16],
        pitch_range=[0, 128],
        drums=False,
        tag="pop",
        tempo=120,
    )
    
    print(f"\nCFG = {cfg_value}")
    sm = generate_from_constraints(constraints, cfg=cfg_value)
    print(f"Generated {sm.note_num()} notes")
    preview_sm(sm)


## Summary

This tutorial demonstrated the constraint functions used in the paper:

**Generation Templates**: Create music from scratch
- `unconditional`: No constraints
- `pentatonic_guitar`: Guitar with pentatonic scale
- `strings_and_flute`: Classical ensemble
- `breakbeat`: Complex drum pattern
- `extreme_drums`: Heavy metal drums
- `triplet_drums`: Triplet-based rhythm

**Re-editing Functions**: Modify regions of existing music
- `replace`: Replace notes in a tick/pitch box
- `retime`: Change timing of notes in a region

**Additive Functions**: Add elements to existing music
- `add_percussion`: Add percussion instruments
- `add_snare_ghost_notes`: Add subtle snare hits
- `add_tom_fill`: Add tom fill in last bars
- `add_locked_in_bassline`: Add bass matching kick drum

Higher constraint CFG values generally result in stronger adherence to the constraints.


### Unconditional Generation
